In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Boolean retrieval with inverted files

## Helper functions

### Tokenizer & Set of Words

In [16]:
from utils import analyzer

print(analyzer.set_of_words("this is a simple test for this function", remove_stopwords = True))
print(analyzer.set_of_words("this is a simple test for this function", remove_stopwords = False))

{'simple', 'test', 'function'}
{'simple', 'function', 'test', 'this', 'is', 'for', 'a'}


## 1st Implementation: using set operations on postings

### The Base Retriever Class

* `n_docs: int`: number of documents added to index
* `documents dict[int, dict{id, vector}]`: collection of documents as dictionary with doc_id as key. Each document is a dictionary with the properties from the dataset and additional properties for the retrieval:
  - `id` hold the document id as generated when loading the document; corresponds to the key in documents
  - `vector` holds the term freqeuncies as dictionary (key=term, value=term frequency)
* `vocabulary: dict[term, int]`: vocabularoy of collection with term as keys and document frequency as values
* `index: dict[term, list[int]]`: inverted index mapping terms to postings. Postings contain doc_id sorted by doc_id

In [17]:
from collections import defaultdict

class BooleanRetriever:
    def __init__(self, collection: list[dict] = None, remove_stopwords: bool = True):
        self.remove_stopwords = remove_stopwords
        self.build_index(collection or [])
    
    def add_document(self, doc: dict):
        self.n_docs += 1
        doc_id = doc['id'] = self.n_docs
        self.documents[doc_id] = doc
        # create vector from str-properties
        text = ' '.join([value for key, value in doc.items() if type(value) == str])
        vector = doc['vector'] = analyzer.set_of_words(text, remove_stopwords = self.remove_stopwords)
        # add to vocabulary and postings
        for term in vector:
            self.vocabulary[term] += 1
            self.index[term].add(doc_id)
    
    def build_index(self, collection: list[dict]):
        self.n_docs = 0
        self.documents = {}
        self.index = defaultdict(set)
        self.vocabulary = defaultdict(int)
        # load all documents
        for doc in collection:
            self.add_document(doc)

### Loading animals data

In [18]:
from utils import table
from datasets import animals as collection

retriever = BooleanRetriever(collection.load())

**Let's have a look at the index**

In [19]:
table.print([collection.format(doc) for doc in retriever.documents.values()], collection.headers(), max_rows = 10)
table.print(sorted([[term, df, retriever.index[term]] for term, df in retriever.vocabulary.items()], key=lambda x: -x[1]), ['term', 'df', 'posting'], max_rows=20)

print(f'{len(retriever.documents)} documents in collection')
print(f'{len(retriever.vocabulary)} distinct terms in collection')
print('{count} postings'.format(count=sum([len(postings) for postings in retriever.index.values()])))

|   id | text                                                                                                 |
|------|------------------------------------------------------------------------------------------------------|
|    1 | dog cat cat ostrich ostrich ostrich bear bear bear bear donkey donkey donkey donkey bee bee bee bee  |
|    2 | cat cat cat cat bear bear bear                                                                       |
|    3 | cat cat cat cat cat horse horse horse horse horse bear bear bear bear bear fly fly fly fly wale wale |
|    4 | rabit rabit rabit rabit rabit tiger tiger tiger                                                      |
|    5 | horse horse horse horse bird bird bird bird bird                                                     |
|    6 | dog dog dog dog dog horse horse horse horse rabit rabit rabit ostrich ostrich                        |
|    7 | fish                                                                                           

## Perform set operations to answer boolean queries
The next section demonstrates how to perform Boolean queries using the inverted index with set operations. It showcases simple queries, AND, OR, AND-NOT operations, complex queries in disjunctive normal form, and arbitrary queries with NOT operations. The results of these operations on the posting sets are printed for each query scenario.

| Boolean Operator | Set Operator for Postings |
| :--- | :--- |
| cat AND dog | `index['cat'] & index['dog']` |
| cat OR dog | `index['cat'] \| index['dog']` |
| cat AND NOT dog | `index['cat'] - index['dog']` |

Using these rules, we can evaluate a wide range of Boolean queries. However, there are some limitations:
- We cannot evaluate OR-queries when one sub-expression is of the form NOT(expr). While it's technically possible to construct NOT(expr) by using all documents except those returned by expr, this approach becomes inefficient for large collections.
- In AND-queries, NOT(expr)-parts need to be re-ordered to the end to apply the `-` set operator. Additionally, at least one element of the AND-query must not be in the form NOT(expr).

Indeed, while these limitations may be viewed as constraints in our implementation, they have minimal impact on practical querying scenarios. Queries like "cat OR NOT(dog)" don't often align with typical search intentions, as they essentially select all documents except those with the condition "dog but not cat", i.e., it can be rephrased as "NOT(dog AND NOT(cat))". 

**Extra challenge: add a parser for Boolean expression and evaluate queries dynamically**

In [20]:
import ipywidgets as widgets

# lets do simple queries by hand for cats, dogs, horses, and birds
cat = retriever.index['cat']
dog = retriever.index['dog']
horse = retriever.index['horse']
bird = retriever.index['bird']

results = {
    'cat': cat,
    'dog': dog,
    'horse': horse,
    'bird': bird,
    'cat AND dog': cat & dog,
    'cat OR dog': cat | dog,
    'horse OR bird': horse | bird,
    'cat AND NOT(dog)': cat - dog,
    'horse AND cat AND NOT(bird)': horse & cat - bird,
    'horse AND cat': horse & cat,
    '(cat AND dog) OR (horse AND cat AND NOT(bird))': (cat & dog) | (horse & cat - bird),
    'horse OR bird': horse | bird,
    '(cat OR dog) AND (horse OR bird)': (cat | dog) & (horse | bird),
    '(cat OR dog) AND NOT(horse OR bird)': (cat | dog) - (horse | bird)
}

queries = {
    'cat AND dog': ['cat', 'dog', 'cat AND dog'],
    'cat OR dog': ['cat', 'dog', 'cat OR dog'],
    'horse OR bird': ['horse', 'bird', 'horse OR bird'],
    'cat AND NOT(dog)': ['cat', 'dog', 'cat AND NOT(dog)'],
    'horse AND cat AND NOT(bird)': ['horse', 'cat', 'bird', 'horse AND cat', 'horse AND cat AND NOT(bird)'],
    '(cat AND dog) OR (horse AND cat AND NOT(bird))': ['cat', 'dog', 'horse', 'bird', 'cat AND dog', 'horse AND cat', 'horse AND cat AND NOT(bird)', '(cat AND dog) OR (horse AND cat AND NOT(bird))'],
    '(cat OR dog) AND (horse OR bird)': ['cat', 'dog', 'cat OR dog', 'horse OR bird', '(cat OR dog) AND (horse OR bird)'],
    '(cat OR dog) AND NOT(horse OR bird)': ['cat', 'dog', 'cat OR dog', 'horse OR bird', '(cat OR dog) AND NOT(horse OR bird)']    
}

def search_boolean_set(query: str):
    table.print([[q, sorted(results[q])] for q in queries[query]], ['query', 'result'])

# interactive selection of scenario
widgets.interact(search_boolean_set, 
    query=widgets.Dropdown(options=list(queries.keys())), 
);

interactive(children=(Dropdown(description='query', options=('cat AND dog', 'cat OR dog', 'horse OR bird', 'ca…

## 2nd Implementation: using stream based evaluation

The set-based evaluation from above does not scale well with the number of documents. In cases with millions of billions of postings for a term, we want to fetch data from an external storage device (which is also a good idea for persistence). But instead of reading all postings into main memory, we read them as streams sorted by the document IDs. Take the postings of cat and dog as an example:

| term | postings|
| :-- | :-- |
| cat | `[1, 4, 8, 10]` |
| dog | `[3, 4, 10, 12]` |

To evaluate a query like "cat AND dog", we fetch the first entry for each term, that is `1` for cat and `3` for dog. If they are the same, we know that the corresponding document fulfills the predicate. If not, then we read the next entry for the term currently having the smallest doc ID. In our example, we read the next cat posting which is `4`. Again, we have no match, so we progress now postings of dog as it currently has the smallest value. The next posting for dog is `4` which matches with the one of cat; hence, we found our first document and return it (in Python we use generators with `yield` as we also do not want to return all results at once but in batches as the user browses through pages). If we need more results to return, we now fetch the next posting for both terms and repeat. Finally, we find `10` and return it as a second answer. If we need more results, we again fetch the next posting for both terms. But since cat does not have more postings, we can terminate the evaluation and stop iteration (dog still has `12` but we already know from the empty cat postings that it cannot match the query). The following visualizes the approach:

|step|cat (next) |dog (next) | action|
|:-- |:-- |:-- |:-- |
| 1 | `1` | `3` | no match, progress cat |
| 2 | `4` | `3` | no match, progress dog |
| 3 | `4` | `4` | match, return `4` as result, wait to provide next result, and progress both cat and dog |
| 4 | `8` | `10` | no match, progress cat |
| 5 | `10` | `10` | match, return `10` as result, wait to provide next result, and progress both cat and dog |
| 6 | - | `12` | stop iteration as all cat postings are visited; remaining postings in dog cannot fulfill predicate |

The OR-operator is implemented similarly, but the iteration returns every time the smallest entry from a sub-expression. For the example above, the OR-operator would first return `1`, progress cat, return `3`, progress dog, return `4`, progress both cat and dog, return `8`, progress cat, return `10`, progress both cat and dog, and finally return `12`. The evaluation stops when all postings are consumed.

The "cat AND NOT(dog)" evaluation progress is the same as with the AND flow but results are different (match if cat != dog):

|step|cat (next) |dog (next) | action|
|:-- |:-- |:-- |:-- |
| 1 | `1` | `3` | match, return `1` as result, wait to provide next result, and progress cat |
| 2 | `4` | `3` | match but cat is not smallest, so we progress dog |
| 3 | `4` | `4` | no match as both have the same value, so we progress both cat and dog |
| 4 | `8` | `10` | no match, return `8` as result, wait to provide next result, and progress cat  |
| 5 | `10` | `10` | no match as both have the same value, so we progress both cat and dog |
| 6 | - | `12` | stop iteration as all cat postings are visited; remaining postings in dog cannot fulfill predicate |

Finally, we can apply the same approach for arbitrary nesting of AND and OR operators since each evaluation scheme above yields document IDs again in a sorted manner. As with atomic term queries, we can only support NOT operators if they are part of an AND expression which at least has one sub-expression without a NOT at the top-level (a nested NOT deeper in the sub-expression is not a problem).

### A representation for Boolean expressions
Next, we build a representation for Boolean expressions that allows us to retrieve arbitrary queries with the stream based evaluation scheme introduced above. Each object in the representation relates to a sub-expression in the query that is evaluated in a stream-based manner. By composition, we can aggregate atomic queries over terms to arbitrarily complex Boolean queries. We introduce a class ``Term`` to denote atomic queries, 2 classes ``And`` and ``Or`` that represent the boolean operators with an arbitrary number of operands (which can be of type ``Term``, ``And``, and ``Or``). Finally, we also define a class ``Not`` that acts on a single sub-expression (again of any sub-type). However, we do not allow nesting of ``Not``, and only ``And`` expression can have a sub-expression with ``Not`` (and must have at leas one subexpression without ``Not``). Our implentation of ``Not`` is to mark sub-expression, but the class has no way to evaluate itself.

#### Atomic term expressions
This class implements a retriever for an atomic term query; it is intentionally kept basic for illustrative purposes. It doesn't involve file reading; rather, it relies on the query parser to pass the postings for the terms. If needed, we can effortlessly replace the for-loop with file reading operations. However, this change introduces complexities because terms may appear multiple times in a Boolean query (e.g., "cat AND dog OR cat AND horse"). Preventing duplicate file reads demands additional buffering logic. Furthermore, for data efficiency, it's advisable to apply compression techniques to reduce the data volume.

In [21]:
class BooleanExpression:
    pass

class Term(BooleanExpression):
    """
        Boolean expression class for atomic term queries
    """
    def __init__(self, term: str, postings: list[int]):
        self.term = term
        self.postings = postings

    def __iter__(self):
        return self.retrieve()
    
    LOG_ACCESS = False

    def retrieve(self):
        for posting in sorted(self.postings):
            if Term.LOG_ACCESS: print(posting)
            yield posting

### Class Not

This is a simple marker class that the And/Or classes are using to implement (or reject) NOT-expressions. Both iterator and generator functions raise an error when invoked to prevent top-level query evaluation of NOT-expressions.

In [22]:
class Not(BooleanExpression):
    """
        Marker class for NOT operator on sub-expression. The retrieve method raises an exception.
        When used during AND operation, the retrieve method of the sub-expression is called.
    """
    def __init__(self, expression):
        self.expression = expression
    
    def __iter__(self):
        return self.retrieve()

    def retrieve(self):
        raise Exception("NOT operator not allowed at top-level of query")

### Class And
This class implements (arbitrary) AND-expression with multiple sub-expressions (can be of any type). This class can handle NOT(expr)-type subexpressions and implements the correct '-' semantics of "cat AND NOT dog". The implementation is kept simple to illustrate the algorithm, and further optimizations are feasible. The implementation first differentiates between positive and negative expressions (for lack of better words). Positive means sub-expressions without top-level NOT-operator, and negative means sub-expressions with a top-level NOT-operator. The `pos_next` and `neg_next` lists contain the next posting for each of the corresponding sub-expressions. Values are `yield`-ed if all 'positive' expressions equal the smallest value over all sub-expressions, and if none of the 'negative' sub-expressions equals that smallest value. The final 2 for-loops progress sub-expressions that have the smallest value as their next value.

In [23]:
class And(BooleanExpression):
    """
        AND-expression with multiple sub-expressions. This operator can handle NOT(expr)-type 
        subexpressions and implements the correct '-' semantics of "cat AND NOT dog".
    """
    def __init__(self, *expressions):
        self.expressions = expressions
        # select expressison that are not Term or that have x._not = False
        self.pos = [e for e in expressions if not isinstance(e, Not)]
        self.neg = [e for e in expressions if isinstance(e, Not)]

    def __iter__(self):
        return self.retrieve()

    def retrieve(self):
        pos_stream = [e.retrieve() for e in self.pos]
        pos_next = [next(e) for e in pos_stream]
        neg_stream = [iter(e.expression.retrieve()) for e in self.neg]
        neg_next = [next(e, None) for e in neg_stream]

        # iterate until one pos_next element is None
        while None not in pos_next:
            # get smallest value from pos_next and neg_next, ignoring None values in neg_next
            smallest = min(pos_next + neg_next, key=lambda x: x if x is not None else float('inf'))
            # check if all entries of pos_next equal smallest, and no entry in neg_next equals smallest
            if all(e is smallest for e in pos_next) and smallest not in neg_next:
                yield smallest
            # for each entry in pos_next and neg_next, fetch next item if entry equals smallest
            for i, e in enumerate(pos_next):
                if e is smallest:
                    pos_next[i] = next(pos_stream[i], None)
            for i, e in enumerate(neg_next):
                if e is smallest:
                    neg_next[i] = next(neg_stream[i], None)

### Class Or
This class implements (arbitrary) OR-expression with multiple sub-expressions (can be of any type). This class **cannot** handle NOT(expr)-type subexpressions and raises an error if a sub-expression has a top-level NOT-operator. The implementation is kept simple to illustrate the algorithm, and further optimizations are feasible. The implementation `yield`s the smallest values found as next posting in its sub-expressions. Then it progresses sub-expressions with that smallest value as their next element. The iteration stops once all sub-expressions are exhausted.

In [24]:
class Or(BooleanExpression):
    """
        OR-expression with multiple sub-expressions. This operator cannot handle NOT(expr)-type subexpressions
    """
    def __init__(self, *expressions):
        # check that there are no NOT(expr)-type subexpressions
        if any(isinstance(e, Not) for e in expressions):
            raise Exception("OR-expression cannot handle NOT(expr)-type subexpressions")
        self.expressions = expressions
    
    def __iter__(self):
        return self.retrieve()

    def retrieve(self):
        iters = [iter(e) for e in self.expressions]
        nexts = [next(e, None) for e in iters]

        while not all(e is None for e in nexts):
            # get smallest value from nexts, ignoring None values
            smallest = min(nexts, key=lambda x: x if x is not None else float('inf'))
            yield smallest
            # for each entry in nexts, fetch next item if entry equals smallest
            for i, e in enumerate(nexts):
                if e is smallest:
                    nexts[i] = next(iters[i], None)

### Parsing boolean queries

We use a simple language in EBNF notation:
```
  expression := term {"OR" term}
  term := factor {"AND" factor}
  factor := <word> | "NOT" factor | "(" expression ")"
```

Given the simple nature of the language, we can use a lexer based on a regular expression such as `\w+|\(|\)` to create tokens from the query string. The parser is manually generated from the EBNF and builds the Boolean expression using the classes introduced above for the current retriever. 

In [11]:
import re

class BooleanRetriever(BooleanRetriever):
    @staticmethod
    def format_bool(expr: BooleanExpression) -> str:
        """
            Prints a Boolean expression in a human-readable format.
        """
        if isinstance(expr, And):
            return '({})'.format(' AND '.join([BooleanRetriever.format_bool(e) for e in expr.expressions]))
        if isinstance(expr, Or):
            return '({})'.format(' OR '.join([BooleanRetriever.format_bool(e) for e in expr.expressions]))
        if isinstance(expr, Not):
            return f"NOT({BooleanRetriever.format_bool(expr.expression)})"
        return f"{expr.term}"
    
    def parse_query(self, query: str) -> BooleanExpression:
        """
            expression := term {"OR" term}
            term := factor {"AND" factor}
            factor := <word> | "NOT" factor | "(" expression ")"
        """
        def factor(tokens: list[str]):
            if not tokens: raise ValueError("parse error")
            if tokens[0] == '(':
                tokens.pop(0)
                expr = expression(tokens)
                if not tokens or tokens[0] != ')': raise ValueError("parse error")
                tokens.pop(0)
                return expr
            if tokens[0] == 'NOT':
                tokens.pop(0)
                return Not(factor(tokens))
            term = tokens.pop(0)
            return Term(term, self.index[term])

        def term(tokens: list[str]):
            expr = [factor(tokens)]
            if not tokens or tokens[0] != 'AND':
                return expr[0]
            while tokens and tokens[0] == 'AND':
                tokens.pop(0)
                expr.append(factor(tokens))
            return And(*expr)    

        def expression(tokens: list[str]):
            expr = [term(tokens)]
            if not tokens or tokens[0] != 'OR': 
                return expr[0]
            while tokens and tokens[0] == 'OR':
                tokens.pop(0)
                expr.append(term(tokens))
            return Or(*expr)

        tokens = re.findall(r"\w+|\(|\)", query)
        return expression(tokens)

### Implementation of the query interface

In [12]:
class BooleanRetriever(BooleanRetriever):
    def search(self, query: str) -> list:
        return self.parse_query(query).retrieve()

In [13]:
retriever = BooleanRetriever(collection.load())

queries = {
    'cat AND dog': ['cat', 'dog', 'cat AND dog'],
    'cat OR dog': ['cat', 'dog', 'cat OR dog'],
    'horse OR bird': ['horse', 'bird', 'horse OR bird'],
    'cat AND NOT(dog)': ['cat', 'dog', 'cat AND NOT(dog)'],
    'horse AND cat AND NOT(bird)': ['horse', 'cat', 'bird', 'horse AND cat', 'horse AND cat AND NOT(bird)'],
    '(cat AND dog) OR (horse AND cat AND NOT(bird))': ['cat', 'dog', 'horse', 'bird', 'cat AND dog', 'horse AND cat', 'horse AND cat AND NOT(bird)', '(cat AND dog) OR (horse AND cat AND NOT(bird))'],
    '(cat OR dog) AND (horse OR bird)': ['cat', 'dog', 'cat OR dog', 'horse OR bird', '(cat OR dog) AND (horse OR bird)'],
    '(cat OR dog) AND NOT(horse OR bird)': ['cat', 'dog', 'cat OR dog', 'horse OR bird', '(cat OR dog) AND NOT(horse OR bird)']    
}

def search_boolean_stream(query: str):
    table.print([[q, list(retriever.search(q))] for q in queries[query]], ['query', 'result'])

# interactive selection of scenario
widgets.interact(search_boolean_stream, 
    query=widgets.Dropdown(options=list(queries.keys())), 
);

interactive(children=(Dropdown(description='query', options=('cat AND dog', 'cat OR dog', 'horse OR bird', 'ca…

### Magic generators

Generators are great to prevent evaluation of results that are not needed. Assume the user is querying with "(cat OR dog) AND NOT(horse OR bird)" which generates a lot of results. Rather than returning hundreds of results at once, a user may want to browse through results page-by-page. Our generators do exactly this; even more, we only read postings that we need to produce the results for each batch returned to the users as they browse through results.

Let's verify this and set `Term.LOG_ACCESS = True`. Every time we fetch a posting, the class Term is printing a line with the term and the next posting. The code below first fetches 5 results, and then, as we imagine that the user moves to the next page, fetches the next 5 results. From the produced output, we see that the evaluation indeed only reads postings as needed.

In [14]:
from itertools import islice

# turn on logging for doc_ids accessed in streams
Term.LOG_ACCESS = True


result = retriever.search('(cat OR dog) AND NOT(horse OR bird)')
# result = retriever.search('cat')
# print(list(result))
print("retrieving first 2 documents for (cat OR dog) AND NOT(horse OR bird)")
print(list(islice(result, 2)))

print("\nretrieving next 2 documents")
print(list(islice(result, 2)))
    

retrieving first 2 documents for (cat OR dog) AND NOT(horse OR bird)
1
4
4
8
16
6
11
[1, 6]

retrieving next 2 documents
8
10
9
13
14
12
13
14
24
17
17
[10, 16]


### What's next
We could extend the code to parse query strings and produce the expressions necessary for the evaluation. We could process real documents, create a document and index dictionary to show real text retrieval. 